In [57]:
import math
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import chi2, f, t

matplotlib.use('TkAgg')  # или 'Qt5Agg'

data = pd.read_excel('data_sber.xlsx')

dff = pd.DataFrame(data)

df = dff.iloc[:, 5:dff.shape[1]]
df = df.dropna()
print(df)

      R_stock    R_imoex      R_usd  R_Inflation  R_IPP    R_f
1    1.356608   0.306052   0.147776         3.20   -9.6   6.14
2    9.029623   6.137722   5.663175         3.40   -9.4   5.67
3    2.044501   1.876307  -0.581583         3.60   -8.0   6.01
4    1.344538  -2.035938   4.757888         3.70   -4.2   6.01
5  -12.285066  -7.406541   2.465188         4.00   -3.6   6.22
6   24.200209  15.498088  -3.768269         4.40   -5.9   6.39
7    8.821055   5.838627  -3.425548         4.90   -1.5   6.52
8   -4.984355  -0.385525   2.611536         5.20    2.1   6.16
9    4.672426   1.976596  -1.854322         5.70   -1.9   6.25
10   7.717363   5.862740   1.877380         5.80   -3.2   6.58
11   2.305683   0.158044  -0.674582         5.50    2.3   7.02
12   4.386525   5.027706  -2.404934         6.00    7.6   7.27
13  -1.396441   3.256663  -0.628177         6.50   12.3   7.17
14  -0.280633  -1.829067   0.229274         6.50   10.0   7.26
15   7.313721   3.751743   0.366382         6.68    7.2

In [58]:

# Заменяем экселевские запятые на точки для чисел если вдруг пандас не сделал этого
for column in df.columns[1:]:  # Start from the second column (index 1)
    df[column] = df[column].astype(str).str.replace(',', '.').astype(float)
print(df)

      R_stock    R_imoex      R_usd  R_Inflation  R_IPP    R_f
1    1.356608   0.306052   0.147776         3.20   -9.6   6.14
2    9.029623   6.137722   5.663175         3.40   -9.4   5.67
3    2.044501   1.876307  -0.581583         3.60   -8.0   6.01
4    1.344538  -2.035938   4.757888         3.70   -4.2   6.01
5  -12.285066  -7.406541   2.465188         4.00   -3.6   6.22
6   24.200209  15.498088  -3.768269         4.40   -5.9   6.39
7    8.821055   5.838627  -3.425548         4.90   -1.5   6.52
8   -4.984355  -0.385525   2.611536         5.20    2.1   6.16
9    4.672426   1.976596  -1.854322         5.70   -1.9   6.25
10   7.717363   5.862740   1.877380         5.80   -3.2   6.58
11   2.305683   0.158044  -0.674582         5.50    2.3   7.02
12   4.386525   5.027706  -2.404934         6.00    7.6   7.27
13  -1.396441   3.256663  -0.628177         6.50   12.3   7.17
14  -0.280633  -1.829067   0.229274         6.50   10.0   7.26
15   7.313721   3.751743   0.366382         6.68    7.2

In [59]:

# Построение диаграммы рассеяния для каждой пары факторов
"""for column in df.columns:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x=column, y='R_stock', color='blue', label='Данные')"""

"for column in df.columns:\n    plt.figure(figsize=(10, 6))\n    sns.scatterplot(data=data, x=column, y='R_stock', color='blue', label='Данные')"

In [60]:
# Функция для удаления выбросов с использованием IQR
def remove_outliers_iqr(df_old):
    for column in df_old.columns:
        Q1 = df_old[column].quantile(0.25)
        Q3 = df_old[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df_old = df_old[(df_old[column] >= lower_bound) & (df_old[column] <= upper_bound)]
    return df_old

# Функция для удаления выбросов с использованием Z-оценки
def remove_outliers_zscore(df_old, threshold=3):
    from scipy import stats
    z_scores = np.abs(stats.zscore(df_old))
    return df[(z_scores < threshold).all(axis=1)]

# Удаление выбросов с использованием IQR
data_no_outliers_iqr = remove_outliers_iqr(df.copy())
print("Данные после удаления выбросов с использованием IQR:")
print(data_no_outliers_iqr.describe())
print(data_no_outliers_iqr)

df = data_no_outliers_iqr.copy()

# Построение диаграммы рассеяния для каждой пары факторов после удаления выбросов
for column in df.columns:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x=column, y='R_stock', color='blue', label='Данные')
    

Данные после удаления выбросов с использованием IQR:
         R_stock    R_imoex      R_usd  R_Inflation      R_IPP        R_f
count  47.000000  47.000000  47.000000    47.000000  47.000000  47.000000
mean    2.044453   1.137492   0.635754     7.145957   2.702128  10.503404
std     7.110625   5.279278   4.220613     2.567428   4.873419   3.537493
min   -12.285066 -10.406081 -10.115196     2.310000  -9.600000   5.670000
25%    -1.175034  -1.994442  -1.638895     5.350000  -0.650000   7.130000
50%     2.305683   1.876307   0.637537     7.440000   3.900000  10.820000
75%     6.589832   4.299920   2.906149     8.680000   5.700000  12.830000
max    18.157216  11.832428   9.365042    12.630000  12.300000  16.840000
      R_stock    R_imoex      R_usd  R_Inflation  R_IPP    R_f
1    1.356608   0.306052   0.147776         3.20   -9.6   6.14
2    9.029623   6.137722   5.663175         3.40   -9.4   5.67
3    2.044501   1.876307  -0.581583         3.60   -8.0   6.01
4    1.344538  -2.035938   4.

In [61]:
df = df.iloc[:, 1:df.shape[1]]
print(df)

# 1) Построение матрицы межфакторных корреляций
correlation_matrix = df.corr()
print("Матрица межфакторных корреляций:")
print(correlation_matrix)

# Визуализация матрицы межфакторных корреляций
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица межфакторных корреляций')
plt.show()

# Определитель матрицы
determinant = np.linalg.det(correlation_matrix)
print(f"\nОпределитель матрицы: {determinant}")

      R_imoex      R_usd  R_Inflation  R_IPP    R_f
1    0.306052   0.147776         3.20   -9.6   6.14
2    6.137722   5.663175         3.40   -9.4   5.67
3    1.876307  -0.581583         3.60   -8.0   6.01
4   -2.035938   4.757888         3.70   -4.2   6.01
5   -7.406541   2.465188         4.00   -3.6   6.22
7    5.838627  -3.425548         4.90   -1.5   6.52
8   -0.385525   2.611536         5.20    2.1   6.16
9    1.976596  -1.854322         5.70   -1.9   6.25
10   5.862740   1.877380         5.80   -3.2   6.58
11   0.158044  -0.674582         5.50    2.3   7.02
12   5.027706  -2.404934         6.00    7.6   7.27
13   3.256663  -0.628177         6.50   12.3   7.17
14  -1.829067   0.229274         6.50   10.0   7.26
15   3.751743   0.366382         6.68    7.2   7.21
16   4.479847  -0.684696         7.40    4.6   6.97
17   1.341363  -3.004583         8.13    6.9   7.09
18  -6.097431   4.974699         8.40    7.4   7.45
19  -2.655895   1.185951         8.39    7.6   8.32
20  -6.78274

In [62]:
# 2) Вычисление статистики Фаррара-Глоубера
n = len(df)  # количество наблюдений
k = df.shape[1]  # количество факторов
alpha = 0.1 # уровень значимости
farrar_glober_statistic = -1*(n-1-1/6*(2*k+5)) * math.log(determinant)
print(f"\nСтатистика Фаррара-Глоубера: {farrar_glober_statistic}")

# Табличное значение для Хи-квадрат
df_chi2 = (k - 1) * (k - 1)  # степени свободы
chi2_table_value = chi2.ppf(1 - alpha, df_chi2)
print(f"\nТабличное значение Хи-квадрат: {chi2_table_value}")


Статистика Фаррара-Глоубера: 21.506601714257553

Табличное значение Хи-квадрат: 23.541828923096105


In [63]:
# 3) Вычисление обратной матрицы
inverse_matrix = np.linalg.inv(correlation_matrix)
print("\nОбратная матрица:")
print(inverse_matrix)

# Визуализация обратной матрицы
plt.figure(figsize=(8, 6))
sns.heatmap(inverse_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Обратная матрица')
plt.show()

# Степени свободы
df1 = k
df2 = n-k-1

# Находим критическое значение F
f_critical = f.ppf(1 - alpha, df1, df2)

print(f"Критическое значение F при уровне значимости {alpha}: {f_critical}")
# Вычисление F-критериев
f_statistics = np.diag(inverse_matrix)
print("\nF-критерии:")
print(f_statistics)

# Визуализация F-критериев
plt.figure(figsize=(8, 4))
sns.barplot(x=df.columns, y=f_statistics)
plt.title('F-критерии')
plt.ylabel('Значение F-критерия')
plt.axhline(y=0, color='r', linestyle='--')  # Уровень 0
plt.show()


Обратная матрица:
[[ 1.0229636   0.07991447  0.11012935  0.07157766  0.01610359]
 [ 0.07991447  1.07254742  0.26227915  0.09167546 -0.03054041]
 [ 0.11012935  0.26227915  1.44663875 -0.01580625 -0.70910657]
 [ 0.07157766  0.09167546 -0.01580625  1.10695053 -0.29884081]
 [ 0.01610359 -0.03054041 -0.70910657 -0.29884081  1.4546411 ]]
Критическое значение F при уровне значимости 0.1: 1.9930359296152707

F-критерии:
[1.0229636  1.07254742 1.44663875 1.10695053 1.4546411 ]


In [64]:
# 4) Вычисление частных коэффициентов корреляции
def partial_correlation(x, y, z):
    """Calculate the partial correlation between x and y controlling for z."""
    model_x = sm.OLS(x, z).fit()
    residual_x = model_x.resid
    model_y = sm.OLS(y, z).fit()
    residual_y = model_y.resid
    return np.corrcoef(residual_x, residual_y)[0, 1]

# Создание матрицы частных корреляций
partial_corr_matrix = pd.DataFrame(index=df.columns, columns=df.columns)
partial_corr_matrix.fillna(0, inplace=True) # убираем нули на всякий случай

for var1 in partial_corr_matrix.index:
    for var2 in partial_corr_matrix.columns:
        if var1 != var2:
            other_vars = df.drop(columns=[var1, var2])
            partial_corr = partial_correlation(df[var1], df[var2], other_vars)
            partial_corr_matrix.loc[var1, var2] = partial_corr

# Заменяем NaN на 0
partial_corr_matrix.fillna(0, inplace=True)

# Вывод матрицы частных корреляций
print("\nМатрица частных корреляций:")
print(partial_corr_matrix)

# Визуализация матрицы частных корреляций
plt.figure(figsize=(8, 6))
sns.heatmap(partial_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица частных корреляций')
plt.show()

ddf = n - k - 1  # Степени свободы

# Функция для расчета t-критерия
def calculate_t_statistic(r, n, k):
    return (r * np.sqrt(n - k - 1)) / np.sqrt(1 - r**2)

# Расчет t-критериев
t_statistics = pd.DataFrame(index=partial_corr_matrix.index, columns=partial_corr_matrix.columns)

for i in range(len(partial_corr_matrix)):
    for j in range(len(partial_corr_matrix)):
        if i != j:
            r = partial_corr_matrix.iloc[i, j]  # Используем iloc для доступа по индексам
            t_statistics.iloc[i, j] = calculate_t_statistic(r, n, k)

print()
print("t-критерии для коэффициентов частной корреляции:")
print(t_statistics)
# Критическое значение t
t_critical = t.ppf(1 - alpha/2, ddf)

print(f"\nКритическое значение t при уровне значимости {alpha}: {t_critical}")

C:\Users\nikit\AppData\Local\Temp\ipykernel_7476\3001287762.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  partial_corr_matrix.fillna(0, inplace=True) # убираем нули на всякий случай
C:\Users\nikit\AppData\Local\Temp\ipykernel_7476\3001287762.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.041064322571481154' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  partial_corr_matrix.loc[var1, var2] = partial_corr
C:\Users\nikit\AppData\Local\Temp\ipykernel_7476\3001287762.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.020916161597449873' has dtype inc


Матрица частных корреляций:
              R_imoex     R_usd  R_Inflation     R_IPP       R_f
R_imoex      0.000000 -0.041064    -0.020916 -0.080606  0.076179
R_usd       -0.041064  0.000000    -0.128950 -0.103301  0.153860
R_Inflation -0.020916 -0.128950     0.000000 -0.021574  0.822160
R_IPP       -0.080606 -0.103301    -0.021574  0.000000  0.229209
R_f          0.076179  0.153860     0.822160  0.229209  0.000000

t-критерии для коэффициентов частной корреляции:
              R_imoex     R_usd R_Inflation     R_IPP       R_f
R_imoex           NaN -0.263162   -0.133958 -0.517813  0.489205
R_usd       -0.263162       NaN   -0.832637 -0.665007  0.997059
R_Inflation -0.133958 -0.832637         NaN -0.138171  9.247833
R_IPP       -0.517813 -0.665007   -0.138171       NaN  1.507798
R_f          0.489205  0.997059    9.247833  1.507798       NaN

Критическое значение t при уровне значимости 0.1: 1.6828780021327079
